<a href="https://colab.research.google.com/github/richardborde/Coursera_Capstone/blob/master/JourneySorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Journey Sorter API

This notebook contains an API for sorting a list of boarding cards for various types of transport.
It will sort the cards in such a way that they form a complete journey from start to end.

### How to Run

1. Execute the cell that installs required libraries.
2. Run the cell that imports libraries.
3. Execute the cells to define the `BoardingCard` and `JourneySorter` classes.
4. Run the cell that reads data from Google Sheets.
5. Execute the sorting algorithm and see the sorted journey.

### Assumptions

- There is always an unbroken chain between all legs of the trip.
- The sorting algorithm has the lowest possible time complexity to sort the cards.


In [80]:
# Import libraries and authenticate the user
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Initialize Google Sheets API Client
creds, _ = default()
gc = gspread.authorize(creds)


In [81]:
class BoardingCard:
    """
    This class represents a boarding card for various types of transport.

    Attributes:
    - transport_type: A string representing the type of transport (e.g., 'Train', 'Bus', etc.)
    - from_city: A string representing the departure city.
    - to_city: A string representing the destination city.
    - seat: A string representing the seat assignment.
    - gate: A string representing the gate number.
    - baggage: A string representing the baggage claim or drop-off information.
    """

    def __init__(self, transport_type, departure, destination, seat, gate, baggage):
        """
        Initializes a new BoardingCard object.

        Args:
        - transport_type (str): The type of transport.
        - from_city (str): The departure city.
        - to_city (str): The destination city.
        - seat (str): The seat assignment.
        - gate (str): The gate number.
        - baggage (str): Baggage claim or drop-off information.

        Returns:
        None
        """
        self.transport_type = transport_type
        self.departure = departure
        self.destination = destination
        self.seat = seat
        self.gate = gate
        self.baggage = baggage


In [82]:
class JourneySorter:
    """
    This class takes a list of boarding cards and sorts them to form a complete journey.
    """

    def __init__(self, boarding_cards):
        """
        Initializes a new JourneySorter object with a list of unsorted boarding cards.
        """
        self.boarding_cards = boarding_cards
        self.sorted_journey = []

    def sort_journey(self):
        """
        Sorts the list of boarding cards to form a complete journey.
        """
        from_to_map = {card.departure: card for card in self.boarding_cards}
        to_from_map = {card.destination: card for card in self.boarding_cards}

        # Find the starting city
        start = None
        for city in from_to_map.keys():
            if city not in to_from_map:
                start = city
                break

        # Sort the journey
        while start:
            card = from_to_map.get(start, None)
            if card:
                self.sorted_journey.append(card)
                start = card.destination
            else:
                break


In [83]:
# Read data from the Google Sheet
worksheet = gc.open_by_key('1WdhXWghzw692Tk2Vncw1oqc7iDgvCANixABYASB4X5w').sheet1
rows = worksheet.get_all_values()

# Convert the rows to BoardingCard objects, taking into account the new headers
boarding_cards = [BoardingCard(*row) for row in rows[1:]]  # Exclude header


In [84]:
print("Loaded Boarding Cards:")
for card in boarding_cards:
    print(card.__dict__)


Loaded Boarding Cards:
{'transport_type': 'flight AA203', 'departure': 'Raleigh', 'destination': 'New York City', 'seat': '4D', 'gate': '3A', 'baggage': '210'}
{'transport_type': 'train Amtrak', 'departure': 'Detroit', 'destination': 'Washington DC', 'seat': '15B', 'gate': '', 'baggage': ''}
{'transport_type': 'bus Greyhound', 'departure': 'Chicago', 'destination': 'Detroit', 'seat': '20', 'gate': '', 'baggage': ''}
{'transport_type': 'flight DL101', 'departure': 'Washington DC', 'destination': 'Atlanta', 'seat': '3C', 'gate': '5B', 'baggage': '105'}
{'transport_type': 'bus Megabus', 'departure': 'Atlanta', 'destination': 'Raleigh', 'seat': '25', 'gate': '', 'baggage': ''}
{'transport_type': 'flight UA789', 'departure': 'San Francisco', 'destination': 'Chicago', 'seat': '12A', 'gate': '7A', 'baggage': '301'}


In [85]:
# Debug print to check if the boarding cards are correctly populated
print("Check initial boarding cards:")
for card in boarding_cards:
    print(f"Transport Type: {card.transport_type}, Departure: {card.departure}, Destination: {card.destination}, Seat: {card.seat}, Gate: {card.gate}, Baggage: {card.baggage}")


Check initial boarding cards:
Transport Type: flight AA203, Departure: Raleigh, Destination: New York City, Seat: 4D, Gate: 3A, Baggage: 210
Transport Type: train Amtrak, Departure: Detroit, Destination: Washington DC, Seat: 15B, Gate: , Baggage: 
Transport Type: bus Greyhound, Departure: Chicago, Destination: Detroit, Seat: 20, Gate: , Baggage: 
Transport Type: flight DL101, Departure: Washington DC, Destination: Atlanta, Seat: 3C, Gate: 5B, Baggage: 105
Transport Type: bus Megabus, Departure: Atlanta, Destination: Raleigh, Seat: 25, Gate: , Baggage: 
Transport Type: flight UA789, Departure: San Francisco, Destination: Chicago, Seat: 12A, Gate: 7A, Baggage: 301


In [86]:
# JourneySorter class definition
class JourneySorter:
    def __init__(self, boarding_cards):
        self.boarding_cards = boarding_cards
        self.sorted_journey = []

    def sort_journey(self):
        # Create mappings
        from_to_map = {card.departure: card for card in self.boarding_cards}
        to_from_map = {card.destination: card for card in self.boarding_cards}

        # Debug prints
        print("From-To Map:", from_to_map)
        print("To-From Map:", to_from_map)

        # Find the starting city
        for city in from_to_map.keys():
          if city not in to_from_map:
            start = city
            break

        # Sort the journey
        current = start
        while current in from_to_map:
            print(f"Current city: {current}")  # Debug line
            self.sorted_journey.append(from_to_map[current])
            current = from_to_map[current].destination
            print(f"Sorted Journey: {[card.destination for card in self.sorted_journey]}")  # Debug line

# Initialize the JourneySorter class and sort the boarding cards
journey_sorter = JourneySorter(boarding_cards)
journey_sorter.sort_journey()

# Display the sorted journey
for i, card in enumerate(journey_sorter.sorted_journey):
    print(f"{i+1}. Take {card.transport_type} from {card.departure} to {card.destination}. "
          f"Sit in seat {card.seat}. Gate: {card.gate}. Baggage: {card.baggage}")


From-To Map: {'Raleigh': <__main__.BoardingCard object at 0x7bdafd267fd0>, 'Detroit': <__main__.BoardingCard object at 0x7bdafd264bb0>, 'Chicago': <__main__.BoardingCard object at 0x7bdafd265ff0>, 'Washington DC': <__main__.BoardingCard object at 0x7bdafd267640>, 'Atlanta': <__main__.BoardingCard object at 0x7bdafd265b40>, 'San Francisco': <__main__.BoardingCard object at 0x7bdafd2676d0>}
To-From Map: {'New York City': <__main__.BoardingCard object at 0x7bdafd267fd0>, 'Washington DC': <__main__.BoardingCard object at 0x7bdafd264bb0>, 'Detroit': <__main__.BoardingCard object at 0x7bdafd265ff0>, 'Atlanta': <__main__.BoardingCard object at 0x7bdafd267640>, 'Raleigh': <__main__.BoardingCard object at 0x7bdafd265b40>, 'Chicago': <__main__.BoardingCard object at 0x7bdafd2676d0>}
Current city: San Francisco
Sorted Journey: ['Chicago']
Current city: Chicago
Sorted Journey: ['Chicago', 'Detroit']
Current city: Detroit
Sorted Journey: ['Chicago', 'Detroit', 'Washington DC']
Current city: Washing